# Scenario comparison

Compare model outputs from the scenario with those from the base.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openmatrix as omx
import tables
from matplotlib import ticker
import nhts

INFLATION_2000_2017 = 1.44

In [ ]:
plt.style.use('asu-light')

In [ ]:
skims = omx.open_file('../model_inputs/skims.omx', 'r')

In [ ]:
dist = np.array(skims['DIST'])
dist = pd.DataFrame(dist, index=np.arange(dist.shape[0]), columns=np.arange(dist.shape[1])).stack()
dist *= 1.609 # begone imperial units

In [ ]:
base_pop = pd.read_csv('../model_inputs/base/persons.csv')
scenario_pop = pd.read_csv('../model_inputs/npv_low_opcost/persons.csv')

In [ ]:
base_hh = pd.read_csv('../model_inputs/base/households.csv')
scenario_hh = pd.read_csv('../model_inputs/npv_low_opcost/households.csv')

In [ ]:
def get_mode_choice(abmfn):
    #abmfn = "/Volumes/Pheasant Ridge/diss_data/model_output/abm/base/pipeline.h5"

    trip_mode_choice = pd.read_hdf(abmfn, "/trips/trip_mode_choice")

    # save some memory
    total_mem_before = trip_mode_choice.memory_usage(deep=True)

    total_mem_before_mb = total_mem_before.sum() // (1024**2)
    trip_mode_choice['primary_purpose'] = trip_mode_choice.primary_purpose.astype('category')
    trip_mode_choice['purpose'] = trip_mode_choice.purpose.astype('category')
    trip_mode_choice['trip_mode'] = trip_mode_choice.trip_mode.astype('category')
    assert not (trip_mode_choice.dtypes == 'object').any()
    total_mem_after = trip_mode_choice.memory_usage(deep=True)  # deep should not matter since no obj columns, but make it definitely comparable
    total_mem_after_mb = total_mem_after.sum() // (1024**2)
    print(f'data type conversion saved {total_mem_before_mb - total_mem_after_mb:,d}mb RAM (before {total_mem_before_mb:,d}mb, now {total_mem_after_mb:,d}mb)')

    # load the tour participation information
    tour_participation = pd.read_hdf(abmfn, '/joint_tour_participants/trip_mode_choice')

    tour_participation

    n_on_tour = tour_participation.groupby('tour_id').size()

    n_on_tour.min()

    trip_mode_choice = trip_mode_choice.merge(pd.DataFrame(n_on_tour.rename('n_on_tour')), left_on='tour_id', right_index=True, how='left')

    # no 1 person tours appear in tour_participation
    trip_mode_choice['n_on_tour'] = trip_mode_choice.n_on_tour.fillna(1)
    
    trip_mode_choice = trip_mode_choice.merge(pd.DataFrame(dist.rename('trip_dist_km')), left_on=['origin', 'destination'], right_index=True, how='left', validate='m:1')
    
    return trip_mode_choice

In [ ]:
base_trips = get_mode_choice('/Volumes/Pheasant Ridge/diss_data/model_output/abm/base/pipeline.h5')

In [ ]:
scenario_trips = get_mode_choice('/Volumes/Pheasant Ridge/diss_data/model_output/abm/npv_low_opcost/pipeline.h5')

In [ ]:
drive_modes = {'DRIVEALONEFREE', 'DRIVEALONEPAY', 'SHARED2FREE', 'SHARED2PAY', 'SHARED3FREE', 'SHARED3PAY'}

In [ ]:
base_drive_trips = base_trips[base_trips.trip_mode.isin(drive_modes)]
scenario_drive_trips = scenario_trips[scenario_trips.trip_mode.isin(drive_modes)]

In [ ]:
# adjust person to vehicle trips
base_vkt_per_capita = np.sum(base_drive_trips.trip_dist_km / base_drive_trips.n_on_tour) / len(base_pop)
scenario_vkt_per_capita = np.sum(scenario_drive_trips.trip_dist_km / scenario_drive_trips.n_on_tour) / len(scenario_pop)

In [ ]:
print(f"""
Base: {base_vkt_per_capita:.4f} vkt/capita/day
Scenario: {scenario_vkt_per_capita:.4f} vkt/capita/day
""")

In [ ]:
# plot for defense presentation
plt.figure(figsize=(6, 2.5))
plt.barh([1, 0], [base_vkt_per_capita, scenario_vkt_per_capita], color=['C0', 'C1'])
plt.yticks([1, 0], ['Base', 'Low operating cost'])
plt.xlabel('Daily vehicle kilometers traveled per capita')
plt.savefig('../../defense/vkt.pdf', bbox_inches='tight')

## What about mode choice?

In [ ]:
# aggregate mode choices in ActivitySim output
mode_choice_map = {
    'BIKE': 'Bike',
    'DRIVEALONEFREE': 'Drive alone',
    'DRIVEALONEPAY': 'Drive alone',
    'DRIVE_COM': 'Transit',
    'DRIVE_EXP': 'Transit',
    'DRIVE_HVY': 'Transit',
    'DRIVE_LOC': 'Transit',
    'DRIVE_LRF': 'Transit',
    'SHARED2FREE': 'Carpool',
    'SHARED2PAY': 'Carpool',
    'SHARED3FREE': 'Carpool',
    'SHARED3PAY': 'Carpool',
    'TAXI': 'Taxi/TNC',
    'TNC_SHARED': 'Taxi/TNC',
    'TNC_SINGLE': 'Taxi/TNC',
    'WALK': 'Walk',
    'WALK_COM': 'Transit',
    'WALK_EXP': 'Transit',
    'WALK_HVY': 'Transit',
    'WALK_LOC': 'Transit',
    'WALK_LRF': 'Transit'
}

base_trips['smplmode'] = base_trips.trip_mode.map(mode_choice_map).astype('category')
scenario_trips['smplmode'] = scenario_trips.trip_mode.map(mode_choice_map).astype('category')
assert not base_trips.smplmode.isnull().any()
assert not scenario_trips.smplmode.isnull().any()

In [ ]:
base_shares = base_trips.groupby('smplmode').size() / len(base_trips)
scenario_shares = (scenario_trips.groupby('smplmode').size() / len(scenario_trips)).reindex(base_shares.index)

In [ ]:
f, ax = plt.subplots()
plt.bar(np.arange(len(base_shares)) - 0.2, base_shares * 100, width=0.4, label='Base')
plt.bar(np.arange(len(base_shares)) + 0.2, scenario_shares * 100, width=0.4, label='Low operating cost')
plt.xticks(np.arange(len(base_shares)), base_shares.index)
plt.legend()
ax.yaxis.set_major_formatter(ticker.FuncFormatter('{:.0f}%'.format))
plt.savefig('../../dissertation/fig/abm/mode_choice_scenario.pdf', bbox_inches='tight')

Ideally we'd also look at this for residents of new housing, but I didn't propagate that information through the population synthesis process.

## Disaggregate by income

In [ ]:
base_hh['income2017'] = base_hh.income * INFLATION_2000_2017
base_hh['inccat'] = pd.cut(base_hh.income2017, [-np.inf, 35000, 50000, 75000, 100000, 150000, 200000, np.inf])
base_trips = base_trips.merge(base_hh[['household_id', 'inccat']], on='household_id', how='left', validate='m:1')
assert not base_trips.inccat.isnull().any()

In [ ]:
scenario_hh['income2017'] = scenario_hh.income * INFLATION_2000_2017
scenario_hh['inccat'] = pd.cut(scenario_hh.income2017, [-np.inf, 35000, 50000, 75000, 100000, 150000, 200000, np.inf])
scenario_trips = scenario_trips.merge(scenario_hh[['household_id', 'inccat']], on='household_id', how='left', validate='m:1')
assert not scenario_trips.inccat.isnull().any()

In [ ]:
# add inccat to pop, we need to calculate vmt/capita
base_pop = base_pop.merge(base_hh, on='household_id', how='left', validate='m:1')
assert not base_pop.inccat.isnull().any()

scenario_pop = scenario_pop.merge(scenario_hh, on='household_id', how='left', validate='m:1')
assert not scenario_pop.inccat.isnull().any()

In [ ]:
# compute vmt by income category
base_vkt_by_income = (
    base_trips[base_trips.trip_mode.isin(drive_modes)].groupby('inccat')
        .apply(lambda df: np.sum(df.trip_dist_km / df.n_on_tour) / np.sum(base_pop.inccat == df.name))
)
scenario_vkt_by_income = (
    scenario_trips[scenario_trips.trip_mode.isin(drive_modes)].groupby('inccat')
        .apply(lambda df: np.sum(df.trip_dist_km / df.n_on_tour) / np.sum(scenario_pop.inccat == df.name))
)

In [ ]:
scenario_vkt_by_income = scenario_vkt_by_income.reindex(base_vkt_by_income.index)

In [ ]:
scenario_vkt_by_income

In [ ]:
f, ax = plt.subplots(figsize=(6, 5))
plt.bar(np.arange(len(base_vkt_by_income)) - 0.2, base_vkt_by_income, width=0.4, label='Base')
plt.bar(np.arange(len(base_vkt_by_income)) + 0.2, scenario_vkt_by_income, width=0.4, label='Low operating cost')
plt.xticks(np.arange(len(base_vkt_by_income)), [
    f'\\${int(c.left):,d}–\\${int(c.right):,d}'
    if np.isfinite(c.left) and np.isfinite(c.right)
    else (f'≤\\${int(c.right):,d}' if np.isfinite(c.right) else f'>\\${int(c.left):,d}') 
    for c in base_vkt_by_income.index], rotation=45, ha='right')
plt.legend()
plt.xlabel('Annual household income (2017 dollars)')
plt.ylabel('Daily VKT per capita')
plt.tight_layout()
#ax.yaxis.set_major_formatter(ticker.FuncFormatter('{:.0f}%'.format))
plt.savefig('../../dissertation/fig/abm/vkt_income_scenario.pdf', bbox_inches='tight')